# Write personal config of your MiniApps

## List of MiniApps

In [ ]:
miniapps = ['onboarding','kobold','script_bakery','scan_prefixes',
            'check_inventory', 'scheduler', 'configmanagement',
            'sync_cmk','sync_phpipam','sync_smokeping','dispatcher',
            'nachtwaechter', 'library_importer']
profiles = ['configmanagement', 'onboarding','kobold','script_bakery','nachtwaechter']
# special cases where we have an additional config (key => app, value=>config_name)
additional = {}
copy_files = {'nachtwaechter': ['index.yaml','mapping.conf.example']}

## Destination directory
The default is to save the configs in your Homedir

In [ ]:
import os
import yaml
BASEDIR = os.path.abspath('')
HOMEDIR = os.path.expanduser('~')

veritas_config_basepath = f'{HOMEDIR}/.veritas/miniapps/'

### We use benedict to store our config

In [ ]:
import os
from benedict import benedict
config = benedict(keyattr_dynamic=True)

# Read config values from file (config_values.yaml)

In [ ]:
with open('config_values.yaml') as f:
    cfg = yaml.safe_load(f.read())
config = benedict(cfg, keyattr_dynamic=True)

# Profile (optional)
You can configure username and password in config_values.yaml<br>
If you want to overwrite these settings use the next cells

## configure username and password to login to your devices

In [ ]:
#config['profile']['username'] = "lab"

## Either set password using variable
This is the encrypted token. If you do not have such a token use the 'getpass' version below.

In [ ]:
#config['profile']['password'] = "encrypted_password"

## or use getpass

In [ ]:
#import getpass
#from veritas.sot import sot as veritas_sot
#password = getpass.getpass(prompt="Enter password: ")
#sot = veritas_sot.Sot()
#auth = sot.auth(salt=config.profile.salt, encryption_key=config.profile.encryptionkey, iterations=int(config.profile.iterations))
#config['profile']['password'] = auth.encrypt(password)

# Internals like further import, settings and internal methods

In [ ]:
from IPython.display import display, HTML
import shutil

In [ ]:
def replace_placeholders(app, app_config, config):
    app_config = app_config.replace(f'__username__', config['profile']['username'])
    app_config = app_config.replace(f'__password__', config['profile']['password'])
    
    # app specific config
    if app in config:
        flatten_config = config[app].flatten(separator="_")
        for key, value in flatten_config.items():
            if isinstance(value, str) and '{HOMEDIR}' in value:
                value = value.replace("{HOMEDIR}", HOMEDIR)
            app_config = app_config.replace(f'__{key}__', str(value))
        
    flatten_config = config.flatten(separator="_")
    for key, value in flatten_config.items():
        if isinstance(value, str) and '{HOMEDIR}' in value:
            value = value.replace("{HOMEDIR}", HOMEDIR)
        app_config = app_config.replace(f'__{key}__', str(value))

    return app_config

In [ ]:
def main_loop(app, filename, veritas_config_basepath):
    config_filename = f'{veritas_config_basepath}/{app}/{filename}.yaml'
    directory = os.path.dirname(config_filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    app_example_config = f'{BASEDIR}/../{app}/conf/{filename}.yaml.example'
    if os.path.isfile(app_example_config):
        with open(app_example_config, "r") as f:
            app_config = f.read()
        new_config = replace_placeholders(app, app_config, config)
        with open(config_filename, "w") as f:
            f.write(new_config)
    else:
        display(f'config {app_example_config} does not exist')

# Now write config

In [ ]:
for app in miniapps:
    main_loop(app, app, veritas_config_basepath)
for app in profiles:
    main_loop(app, 'profiles', veritas_config_basepath)
    main_loop(app, 'salt', veritas_config_basepath)
for app in additional:
    main_loop(app, additional[app], veritas_config_basepath)

# And copy some additional files

In [ ]:
for app in copy_files:
    for filename in copy_files[app]:
        soure = f'{BASEDIR}/../{app}/conf/{filename}'
        destination = f'{veritas_config_basepath}/{app}/{filename}'
        shutil.copy(soure, destination, follow_symlinks=True)